In [20]:
import time
start_time = time.time()
import cv2
import numpy as np
import math
from PIL import Image
from Cryptodome.Cipher import AES
%matplotlib auto
image = cv2.imread('bird.jpg',1)
cv2.imshow('original', image)

Using matplotlib backend: Qt5Agg


In [21]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [22]:
key = b'aespasswordisnow'
cipher_a = AES.new(key, AES.MODE_EAX)
nonce = cipher_a.nonce
ciphertext_enc, tag = cipher_a.encrypt_and_digest(img_str)

In [23]:
cipher_a = AES.new(key, AES.MODE_EAX, nonce=nonce)
plaintext_enc = cipher_a.decrypt(ciphertext_enc)

In [24]:
img_str_np = np.frombuffer(plaintext_enc, dtype=np.uint8)

In [25]:
img = cv2.imdecode(img_str_np, 1)

In [26]:
cv2.imshow('decrypted', img)

In [27]:
#img_str2 = cv2.imencode('.jpg', img)[1].tobytes()
#img_str2 = img_str2[0:len(img_str)]
#img_str_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [28]:
def nearest_square(limit):
    next = math.floor(math.sqrt(limit)) + 1
    return next * next

In [29]:
img_str_np_enc = np.frombuffer(ciphertext_enc, dtype=np.uint8)
#add elements to the end of the array to make it a square
q = nearest_square(len(img_str_np_enc))

while len(img_str_np_enc) != q:
    img_str_np_enc = np.append(img_str_np_enc, 0)
    

img_str_np_enc1 = img_str_np_enc.reshape(int(math.sqrt(len(img_str_np_enc))), int(math.sqrt(len(img_str_np_enc))))

In [30]:
enc_img = Image.fromarray(img_str_np_enc1)
print(img_str_np_enc1.shape)

(110, 110)


In [31]:
enc_img.show()

In [32]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(255 / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [33]:
cv2.PSNR(image, img)

42.722213649793986

In [34]:
MSE = np.square(np.subtract(img_str_np,og_img_np)).mean()
MSE

0.0

In [35]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [36]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [37]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [38]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.706216335296631 seconds ---
